<a href="https://colab.research.google.com/github/jiheddachraoui/occupancy_grid_generator/blob/main/SG_fusion_saver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREPARATION

In [ ]:
import os
import sys
import csv
import imageio
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
from collections import Counter
import tensorflow as tf
from sklearn.linear_model import RANSACRegressor
from scipy import stats
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [ ]:
datadir = '/content/drive/MyDrive/s_data/'
scenario_file='scenario_test2_gt/'
scenario='ground truth/'+scenario_file
output='output/'+scenario_file


for item in os.listdir(os.path.join(datadir,scenario)):
    # Construct the full path to the item
    item_path = os.path.join(os.path.join(datadir,scenario), item)
    # Check if the item is a directory
    if os.path.isdir(item_path) and item != '.ipynb_checkpoints':
        # Print the path of the subdirectory
        print(item_path)



/content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/hero vehicle.lincoln.mkz_2017
/content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/adversary vehicle.tesla.model3


In [ ]:
REDUCED_LABEL_COLORS = np.array([
    (255, 255, 255), # None
    (128, 64, 128),  # Road RoadLines
    (80, 80, 80),    # Building,Wall, static ,Fences
    (250, 170, 30),    # Other ,GuardRail ,RailTrack ,15Bridge
    (110, 190, 160), # Pole ;TrafficLight; Static
    (107, 142, 35),  # Vegetation
    (0, 0, 142),     # Vehicle Dynamic pedestrians
    (244, 35, 232),  # Sidewalk Ground Terrain
    (70, 130, 180),   # Water

]) / 255.0 # normalize each channel [0-1] since is what Open3D uses

In [ ]:
ALPHA = 1
BHETA = 1*np.pi/180
RESOLUTION = 0.1
MAX_RANGE = 70
MAP_WIDTH = 100
SPHERICAL2CARTESIAN_BIAS = 6
### Some derived parameters
# OOR_MASK = A mask to filter out measurements that are out of MAX_RANGE
MAP_SIZE_X = int(MAP_WIDTH/RESOLUTION)
MAP_SIZE_Y = int(MAP_WIDTH/RESOLUTION)
xarr = np.arange(-MAP_WIDTH/2,MAP_WIDTH/2,RESOLUTION)
yarr = np.arange(-MAP_WIDTH/2,MAP_WIDTH/2,RESOLUTION)
MAP_XX, MAP_YY = np.meshgrid(xarr, -yarr)
rgrid = np.sqrt(np.add(np.square(MAP_XX),np.square(MAP_YY)))
OOR_MASK = rgrid >= MAX_RANGE
DYNAMIC_THRESHOLD=0.5
num_classes=9
FREE_CONF = 0.7
OCC_CONF = 0.7
DYNAMIC_THRESHOLD = 0.4
sgm_time_0 = np.zeros((MAP_SIZE_Y,MAP_SIZE_X,num_classes))
sgm_time_0[:,:,0] = 1

In [ ]:
def load_matrix(folder_path,timestamp):
  ts = str(timestamp)
  for file_name in os.listdir(folder_path):
          if os.path.splitext(file_name)[0] == ts:

              file_path = os.path.join(folder_path,file_name)
              #print(file_path)
              break

  m=np.load(file_path)
  return m

In [ ]:
def filter_road_points(lidar_in, is_road, threshold=0.5):
        y_true = lidar_in[:, 1]
        y = np.zeros_like(y_true)
        y = y_true * is_road  # set road points' predicted height to true height
        delta_y = np.absolute(y_true-y).flatten()
        is_not_road = delta_y > threshold
        lidar_out = lidar_in[is_not_road, :].copy()
        return lidar_out


In [ ]:
L0_CONF = 0.1
L1_CONF = 0.8
L2_CONF = 1
L3_CONF = 1
L4_CONF = 1
L5_CONF = 1
L6_CONF = 1
L7_CONF = 1
L8_CONF = 1



class_conf={
    0 : L0_CONF,
    1 : L1_CONF,
    2 : L2_CONF,
    3 : L3_CONF,
    4 : L4_CONF,
    5 : L5_CONF,
    6 : L6_CONF,
    7 : L7_CONF,
    8 : L8_CONF}



def generate_semantic_dgm(lidar_in, ogm_shape):
  ### Calculate the position of LiDAR points in spherical coordinate
    rphi_meas = np.zeros((lidar_in.shape[0], 3))
    rphi_meas[:, 1] = np.sqrt(np.add(np.square(lidar_in[:, 0]), np.square(lidar_in[:, 1]))) / ALPHA
    rphi_meas[:, 0] = (np.arctan2(lidar_in[:, 1], lidar_in[:, 0]) + np.pi) / BHETA
    rphi_meas[:, 2]=lidar_in[:,2]


    rphi_meas = np.unique(rphi_meas.astype(int), axis=0)

    rphi_meas = rphi_meas[rphi_meas[:, 1] < int(MAX_RANGE / ALPHA), :]
    rphi_meas = rphi_meas[rphi_meas[:, 0] < int(2 * np.pi / BHETA), :]

    ### Initiate and fill the spherical scan grid
    sg_ang_bin = int(2 * np.pi / BHETA)
    sg_rng_bin = int(MAX_RANGE / ALPHA)
    # Initiation (Condition 3)

    label_grid_np = np.zeros((sg_ang_bin, sg_rng_bin,num_classes))
    label_grid_np[:,:,0] = 1
    label_grid = [[[] for _ in range(sg_rng_bin)] for _ in range(sg_ang_bin)]

    # Condition 1

    for i in range(rphi_meas.shape[0]):

      ang = rphi_meas[i, 0]
      rng = rphi_meas[i, 1]
      label = rphi_meas[i, 2]
      #label_grid[int(ang), int(rng)] = label
      label_grid[ang][rng].append(label)

    # Assign majority label to each cell
    for ang in range(sg_ang_bin):
        for rng in range(sg_rng_bin):
            cell_labels = label_grid[ang][rng]

            # Exclude label 0 from the list of cell_labels
            #cell_labels = [label for label in cell_labels if label != 0]


            if len(cell_labels) > 0:
                # Count the occurrences of each label
                  label_counts = Counter(cell_labels)
                  label_counts = {label: label_counts[label] * class_conf.get(label, 1) for label in label_counts}
                  total_count = sum(label_counts.values())

                  # Calculate the appearance percentage for each label in cell_labels
                  appearance_percentages = {label: count / total_count for label, count in label_counts.items()}

                  # Fill in the remaining labels in label_counts with 0 counts
                  for label in range(num_classes):
                      if label not in label_counts:
                          label_counts[label] = 0

                  appearance_tuple = tuple(appearance_percentages.get(label, 0) for label in range(num_classes))
                  #print(appearance_tuple)
                  # Assign the appearance tuple to the grid cell
                  label_grid_np[ang][rng] = appearance_tuple
            else:

              label_grid_np[ang][rng] = (1,0,0,0,0,0,0,0,0)
    center = sg_ang_bin // 2
    for ang in range(sg_ang_bin):
        ang_arr = rphi_meas[rphi_meas[:, 0] == ang, 1]

        if len(ang_arr) == 0:

            label_grid_np[ang, :] = (0.3,0.7,0,0,0,0,0,0,0)
        else:

            min_r = np.min(ang_arr)

            label_grid_np[ang, :(min_r)] = (0.3,0.7,0,0,0,0,0,0,0)

            #label_grid_np[ang, :3] = (0.2, 0.6, 0, 0, 0, 0 ,0.2, 0, 0)


    ### Convert the spherical scan grid to the cartesian one
    ogm_sz = (ogm_shape[1], ogm_shape[0])
    ogm_cen = (int(ogm_shape[1] / 2), int(ogm_shape[0] / 2))
    radius = (MAX_RANGE / RESOLUTION) + SPHERICAL2CARTESIAN_BIAS

    label_step = cv2.warpPolar(label_grid_np, ogm_sz, ogm_cen, radius, cv2.WARP_INVERSE_MAP)

    label_step[OOR_MASK] = (1,0,0,0,0,0,0,0,0)
    for i in range(label_step.shape[-1]):
        label_step[:, :, i] = cv2.rotate(label_step[:, :, i], cv2.ROTATE_90_CLOCKWISE)
    #label_step = cv2.rotate(label_step, cv2.ROTATE_90_CLOCKWISE)


    return label_step

In [ ]:

# Update the SGM with DST rule of combination
def update_sgm(prior_dgm, new_dgm):
    num_classes = new_dgm.shape[2]

    conflict_mass = np.zeros_like(new_dgm[:, :, 0])
    class_mass = np.zeros_like(new_dgm[:, :, 1:])



    for i in range(1,num_classes):
        for j in range(1,num_classes):

            if i != j:
                conflict_mass += np.multiply(prior_dgm[:, :, i], new_dgm[:, :, j])
                conflict_mass += np.multiply(prior_dgm[:, :, j], new_dgm[:, :, i])

        class_mass[:, :, i-1] += np.multiply(prior_dgm[:, :, 0], new_dgm[:, :, i])
        class_mass[:, :, i-1] += np.multiply(prior_dgm[:, :, i], new_dgm[:, :, 0])
        #class_mass[:, :, i-1] = np.divide(class_mass[:, :, i-1], np.ones_like(conflict_mass) - conflict_mass +1e-10)  #DS
        class_mass[:, :, i-1] += conflict_mass #yager
        #class_mass[:, :, i-1] += np.divide(class_mass[:, :, i-1], np.ones_like(conflict_mass) - conflict_mass +1e-10) #perso


    #class0 is the unknown mass
    unknown_mass = np.multiply(prior_dgm[:,:,0],new_dgm[:,:,0])
    #unknown_mass = np.divide(unknown_mass,np.ones_like(conflict_mass) - conflict_mass+1e-10) #DS
    unknown_mass += conflict_mass #yager
    #unknown_mass = +np.divide(unknown_mass,np.ones_like(conflict_mass) - conflict_mass+1e-10) #perso
    updated_dgm = np.dstack((unknown_mass,class_mass))

    for i in range(prior_dgm.shape[0]):
      for j in range(prior_dgm.shape[1]):
          if np.array_equal(prior_dgm[i, j], new_dgm[i, j]):
              # If the belief values at (i, j) are the same element-wise, assign the values to the masses
              updated_dgm[i, j] = new_dgm[i, j]
              continue


    return updated_dgm, conflict_mass

# Convert the SGM to a displayable figure
def predict_sgm(dgm, dynamic_mass,DYNAMIC_THRESHOLD=0.4):

    max_mass = np.argmax(dgm, axis=2)

    pred_map = np.zeros(dgm[:,:,:3].shape)

    #print(np.unique(max_mass))
    for i in range(num_classes):
        pred_map[max_mass == i] = REDUCED_LABEL_COLORS[i] * 255

    # The dynamic cells:
    pred_map[dynamic_mass >= DYNAMIC_THRESHOLD] = REDUCED_LABEL_COLORS[6] * 255

    return pred_map.astype(np.uint8)


In [ ]:
def update_sgm_scenario(*new_dgms):
    num_classes = new_dgms[0].shape[2]
    num_inputs = len(new_dgms)

    conflict_mass = np.zeros_like(new_dgms[0][:, :, 0])
    class_mass = np.zeros_like(new_dgms[0][:, :, 1:])
    unknown_mass = np.zeros_like(new_dgms[0][:, :, 0])

    for i in range(num_inputs - 1):
        for j in range(i + 1, num_inputs):
            for k in range(1, num_classes):
                conflict_mass += np.multiply(new_dgms[i][:, :, k], new_dgms[j][:, :, k])
                conflict_mass += np.multiply(new_dgms[j][:, :, k], new_dgms[i][:, :, k])

            for k in range(1, num_classes):
                class_mass[:, :, k-1] += np.multiply(new_dgms[i][:, :, 0], new_dgms[j][:, :, k])
                class_mass[:, :, k-1] += np.multiply(new_dgms[i][:, :, k], new_dgms[j][:, :, 0])
                #class_mass[:, :, k-1] = np.divide(class_mass[:, :, k-1], np.ones_like(conflict_mass) - conflict_mass + 1e-10)#DS
                #class_mass[:, :, k-1] += conflict_mass #yager
            unknown_mass *= np.multiply(unknown_mass, new_dgms[j][:, :, 0])
            #unknown_mass = np.divide(unknown_mass,np.ones_like(conflict_mass) - conflict_mass+1e-10) #DS
            #unknown_mass += conflict_mass #yager
    updated_dgm = np.dstack((unknown_mass, class_mass))

    for i in range(new_dgms[-1].shape[0]):
        for j in range(new_dgms[-1].shape[1]):
            if np.array_equal(new_dgms[-1][i, j], updated_dgm[i, j]):
                updated_dgm[i, j] = new_dgms[-1][i, j]

    return updated_dgm, conflict_mass

In [ ]:
# Shift the map according the vehicle's poses
def shift_pose_sgm(dgm, init, fin):
  dgm_o = dgm.copy()
  #theta = init[2] /180 * np.pi
  theta = -init[2]
  rot_m = np.array([[np.cos(theta),np.sin(theta)],[-np.sin(theta),np.cos(theta)]])
  trs_m = np.array([[init[0]],[init[1]]])
  point = np.array(fin[:2]).reshape((-1,1))
  point_1 = (point - trs_m)
  point_2 = np.dot(rot_m,-point_1)
  delta_theta = (fin[2] - init[2])
  delta = np.array([point_2[1,0]/RESOLUTION,point_2[0,0]/RESOLUTION,0])



  M = np.array([[1,0,delta[0]],[0,1,-delta[1]]])
  dst = cv2.warpAffine(dgm_o,M,(dgm_o.shape[1],dgm_o.shape[0]))
  M = cv2.getRotationMatrix2D((dgm_o.shape[1]/2+0.5,dgm_o.shape[0]/2+0.5),-delta_theta* 180 / np.pi,1)
  #M = cv2.getRotationMatrix2D((dgm_o.shape[1]/2+0.5,dgm_o.shape[0]/2+0.5),delta_theta,1)
  dst = cv2.warpAffine(dst,M,(dgm_o.shape[1],dgm_o.shape[0]))

  search_value = (0, 0, 0, 0, 0, 0, 0, 0, 0)
  border_value = (1, 0, 0, 0, 0, 0, 0, 0, 0)

  # Create a mask for pixels with the search value
  mask = np.all(dst == search_value, axis=-1)

  # Replace the pixels with the replace value
  dst[mask] = border_value

  return dst

In [ ]:
vehicles=[]
for item in os.listdir(os.path.join(datadir,scenario)):
    # Construct the full path to the item
    if os.path.isdir(item_path) and item != '.ipynb_checkpoints':
        folder_path = os.path.join(os.path.join(datadir,scenario), item)
        output_path = os.path.join(datadir,output)
        lidar_path=folder_path+'/Lidar'
        img_path=folder_path+'/bird_eye'
        save_path=output_path+'SG fusion'
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        num_files = len([f for f in os.listdir(lidar_path) if os.path.isfile(os.path.join(lidar_path, f))])
        print('Number of files in {} :'.format(lidar_path), num_files)
        vehicles.append((folder_path,num_files,save_path))

Number of files in /content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/hero vehicle.lincoln.mkz_2017/Lidar : 406
Number of files in /content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/adversary vehicle.tesla.model3/Lidar : 406


In [ ]:
vehicles

[('/content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/hero vehicle.lincoln.mkz_2017',
  406,
  '/content/drive/MyDrive/s_data/output/scenario_test2_gt/SG fusion'),
 ('/content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/adversary vehicle.tesla.model3',
  406,
  '/content/drive/MyDrive/s_data/output/scenario_test2_gt/SG fusion')]

In [ ]:
folder_paths=[]

for i, vehicle in enumerate(vehicles, start=1):
    variable_name = f'folder_path{i}'
    folder_path = os.path.join(datadir, scenario, vehicle[0])
    globals()[variable_name] = folder_path
    folder_paths.append(folder_path)
nbr_frames=[]
for i, vehicle in enumerate(vehicles, start=1):

    nbr = vehicle[1]

    nbr_frames.append(nbr)
nbr=min(nbr_frames)

In [ ]:
folder_paths

['/content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/hero vehicle.lincoln.mkz_2017',
 '/content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/adversary vehicle.tesla.model3']

In [ ]:
'''
import random
seed_value = 13

# Set the seed for the random number generator
random.seed(seed_value)
# Randomly permute the order of folder_paths
random.shuffle(folder_paths)
print(folder_paths)'''

'\nimport random\nseed_value = 13\n\n# Set the seed for the random number generator\nrandom.seed(seed_value)\n# Randomly permute the order of folder_paths\nrandom.shuffle(folder_paths)\nprint(folder_paths)'

In [ ]:
folder_paths

['/content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/hero vehicle.lincoln.mkz_2017',
 '/content/drive/MyDrive/s_data/ground truth/scenario_test2_gt/adversary vehicle.tesla.model3']

In [ ]:
print(nbr)

406


In [ ]:
for date in range(nbr):

  for i, folder_path in enumerate(folder_paths, start=1):

      save_path=vehicles[i-1][2]+'/fusion_image'
      save_path1=vehicles[i-1][2]+'/mass_function'
      save_path2=vehicles[i-1][2]+'/conflict'
      if not os.path.exists(save_path):
        os.makedirs(save_path)
      if not os.path.exists(save_path1):
        os.makedirs(save_path1)
      if not os.path.exists(save_path2):
        os.makedirs(save_path2)
      transformation_matrices_path = os.path.join(folder_path, "pos_matrices/lidar_2_world")
      transformation_matrices_path_variable = f'transformation_matrices_path{i}'
      globals()[transformation_matrices_path_variable] = transformation_matrices_path



      transformation_matrix_variable = f'transform{i}'
      globals()[transformation_matrix_variable] = load_matrix(transformation_matrices_path, date)
      transform = globals()[transformation_matrix_variable]


      location_variable = f'loc{i}'
      globals()[location_variable] = transform[:3, 3:].flatten().tolist()

      rotation_path = os.path.join(folder_path, "pos_matrices/rotation_pyr")
      rotation_path_variable = f'rot_path{i}'
      globals()[rotation_path_variable] = rotation_path

      rotation_variable = f'rot{i}'
      globals()[rotation_variable] = load_matrix(rotation_path, date)



      R = np.array(transform[:3, :3])
      pitch = np.arcsin(R[2, 0])
      yaw = -np.arctan2(R[1, 0], R[0, 0])
      roll = -np.arctan2(R[2, 1], R[2, 2])
      pose_variable = f'pose{i}'
      globals()[pose_variable] = np.array([globals()[location_variable][0], globals()[location_variable][1], yaw])


      lidar_variable = f'lidar{i}'
      lidar_path = os.path.join(folder_path, "Lidar")
      lidar_file = os.path.join(lidar_path, f'{date}.npy')

      lidar_data = np.load(lidar_file)
      globals()[lidar_variable] = lidar_data.reshape(-1, 6)

      lidar_raw_variable = f'lidar_raw{i}'
      globals()[lidar_raw_variable] = globals()[lidar_variable].reshape(-1, 6)

      globals()[lidar_raw_variable] = globals()[lidar_raw_variable][globals()[lidar_raw_variable][:, 2] <= 0, :]

      radius = 2.7
      distances_variable = f'distances{i}'
      distances = np.sqrt((globals()[lidar_raw_variable][:, 0])**2 + (globals()[lidar_raw_variable][:, 1])**2)
      globals()[distances_variable] = distances

      globals()[lidar_raw_variable] = globals()[lidar_raw_variable][globals()[distances_variable] > radius, :]
      labels_variable = f'labels{i}'
      labels = np.array(globals()[lidar_raw_variable][:, 5]).astype(int)
      globals()[labels_variable] = labels

      lidar_labeled_variable = f'lidar_labeled{i}'
      lidar_labeled = np.array(globals()[lidar_raw_variable][..., [0, 1, 2, 5]])
      globals()[lidar_labeled_variable] = lidar_labeled

      globals()[lidar_raw_variable] = globals()[lidar_raw_variable][:, :3]

      road_filter_variable = f'road_filter{i}'
      road_filter = np.zeros_like(globals()[labels_variable])
      road_filter[(globals()[labels_variable] == 0) | (globals()[labels_variable] == 7) | (globals()[labels_variable] == 6)] = 1
      globals()[road_filter_variable] = road_filter

      lidar_nonroad_variable = f'lidar_nonroad{i}'
      globals()[lidar_nonroad_variable] = filter_road_points(globals()[lidar_raw_variable], globals()[road_filter_variable], 0.1)


      reduced_labels_variable = f'reduced_labels{i}'

      lidar_rlabeled_variable = f'lidar_rlabeled{i}'
      lidar_nonroad_rlabeled_variable = f'lidar_nonroad_rlabeled{i}'


      globals()[reduced_labels_variable] = np.zeros_like(globals()[labels_variable])
      globals()[reduced_labels_variable][(globals()[labels_variable] == 7) | (globals()[labels_variable] == 6)] = 1
      globals()[reduced_labels_variable][(globals()[labels_variable] == 1) | (globals()[labels_variable] == 2) | (globals()[labels_variable] == 11) | (globals()[labels_variable] == 19)] = 2
      globals()[reduced_labels_variable][(globals()[labels_variable] == 3) | (globals()[labels_variable] == 15) | (globals()[labels_variable] == 17) | (globals()[labels_variable] == 18)] = 3
      globals()[reduced_labels_variable][(globals()[labels_variable] == 5) | (globals()[labels_variable] == 18) | (globals()[labels_variable] == 19)] = 4
      globals()[reduced_labels_variable][(globals()[labels_variable] == 9)] = 5
      globals()[reduced_labels_variable][(globals()[labels_variable] == 4) | (globals()[labels_variable] == 10) | (globals()[labels_variable] == 20)] = 6
      globals()[reduced_labels_variable][(globals()[labels_variable] == 8) | (globals()[labels_variable] == 14) | (globals()[labels_variable] == 22)] = 7
      globals()[reduced_labels_variable][(globals()[labels_variable] == 21)] = 8


      globals()[lidar_rlabeled_variable] = np.concatenate((globals()[lidar_raw_variable], globals()[reduced_labels_variable][..., np.newaxis]), axis=-1)
      road_filter_variable = f'road_filter{i}'
      globals()[lidar_nonroad_rlabeled_variable] = filter_road_points(globals()[lidar_rlabeled_variable], globals()[road_filter_variable], 0.1)


      lidar_sgm_variable = f'lidar_sgm{i}'

      lidar_nonroad_rlabeled = globals()[lidar_nonroad_rlabeled_variable]
      globals()[lidar_sgm_variable] = lidar_nonroad_rlabeled[:, [0, 1, 3]]

      sgm_variable = f'sgm{i}'

      globals()[sgm_variable] = generate_semantic_dgm(globals()[lidar_sgm_variable], sgm_time_0.shape)


      sgm_time_variable = f'sgm_time_{i}'
      dynamic_mass_variable = f'dynamic_mass_{i}'
      sgm_pred_variable = f'sgm_pred_{i}'

      globals()[sgm_time_variable], globals()[dynamic_mass_variable] = update_sgm(sgm_time_0, globals()[sgm_variable])
      globals()[sgm_pred_variable] = predict_sgm(globals()[sgm_time_variable], globals()[dynamic_mass_variable],0.7)

  shift_sgm_time = [shift_pose_sgm(sgm_time_1, pose1, pose2)]

  if 'pose3' in globals():
      shift_sgm_time.append(shift_pose_sgm(sgm_time_3, pose3, pose2))

  if 'pose4' in globals():
      shift_sgm_time.append(shift_pose_sgm(sgm_time_4, pose4, pose2))

  sgm_fuse, dynamic_mass_fuse = update_sgm_scenario(sgm_time_2, *shift_sgm_time)
  sgm_pred_fuse = predict_sgm(sgm_fuse, dynamic_mass_fuse, 100)
  '''plt.imshow(sgm_pred_fuse)
  plt.show()'''
  rgb_image = cv2.cvtColor(sgm_pred_fuse, cv2.COLOR_BGR2RGB)
  cv2.imwrite(save_path+'/{}.png'.format(date), rgb_image)
  #imageio.imwrite(save_path+'/{}.jpg'.format(date),sgm_pred_fuse)
  print(save_path+'/{}.png'.format(date))
  np.save(save_path1+'/{}.npy'.format(date),sgm_fuse)
  print(save_path1+'/{}.npy'.format(date))
  np.save(save_path2+'/{}.npy'.format(date),dynamic_mass_fuse)
  print(save_path2+'/{}.npy'.format(date))

/content/drive/MyDrive/s_data/output/scenario_test2_gt/SG fusion/fusion_image/0.png
/content/drive/MyDrive/s_data/output/scenario_test2_gt/SG fusion/mass_function/0.npy
/content/drive/MyDrive/s_data/output/scenario_test2_gt/SG fusion/conflict/0.npy
